In [1]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import string

import nltk
from transformers import BertTokenizer

In [2]:
df = pd.read_csv('data/url_10_Ks.csv.gz', compression='gzip')

In [3]:
display(df)

,date,filingDate,url,cik,ticker
0,2024,2024-01-29,https://www.sec.gov/Archives/edgar/data/131860...,1318605,TSLA
1,2023,2023-01-31,https://www.sec.gov/Archives/edgar/data/131860...,1318605,TSLA
2,2022,2022-02-07,https://www.sec.gov/Archives/edgar/data/131860...,1318605,TSLA
3,2021,2021-02-08,https://www.sec.gov/Archives/edgar/data/131860...,1318605,TSLA
4,2020,2020-02-13,https://www.sec.gov/Archives/edgar/data/131860...,1318605,TSLA
...,...,...,...,...,...
51739,2023,2023-02-24,https://www.sec.gov/Archives/edgar/data/106868...,1068689,ATDS
51740,2022,2022-03-31,https://www.sec.gov/Archives/edgar/data/106868...,1068689,ATDS
51741,2021,2021-03-23,https://www.sec.gov/Archives/edgar/data/106868...,1068689,ATDS
51742,2020,2020-04-17,https://www.sec.gov/Archives/edgar/data/106868...,1068689,ATDS


In [4]:
HEADER= {
    'accept-ranges': 'bytes',
    'content-encoding': 'gzip',
    'content-length': '114345',
    'content-type': 'text/html',
    'server': 'AmazonS3',
    'strict-transport-security': 'max-age=31536000 ; includeSubDomains ; preload',
    'vary': 'Accept-Encoding',
    'x-amz-id-2': 'Ytug3pAfKASKCbpl2To2zYpYEiyXnVX+yC02wqq3fhvZNVqp0enNpCaM1eWyNIjh6cSRlPfgDt4=',
    'x-amz-replication-status': 'REPLICA',
    'x-amz-request-id': 'F05XSP9XFT6H683W',
    'x-amz-version-id': 'zeLgRklys7MpRbnb0l56tUJBC33qVCRj',
    'x-content-type-options': 'nosniff',
    'x-frame-options': 'SAMEORIGIN',
    'x-xss-protection': '1; mode=block',
    'authority': 'www.sec.gov',
    'method': 'GET',
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-encoding': 'gzip, deflate, br, zstd',
    'accept-language': 'fr-CH,fr-FR;q=0.9,fr;q=0.8,en-US;q=0.7,en;q=0.6,de;q=0.5',
    'cache-control': 'max-age=0',
    'priority': 'u=0, i',
    'referer': 'http://localhost:8888/',
    'sec-ch-ua': '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
}


In [5]:
def get_paragraphs_from_url(url):
    for i in range(20):
        try:
            # Send an HTTP GET request to the URL
            response = requests.get(url, headers=HEADER)
                                    
            response.raise_for_status()  # Raise an exception for HTTP errors
    
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')
    
            # Extract the treated text from the HTML (i.e., the text without HTML tags)
            text = soup.get_text()
    
            paragraphs = text.split('\n\n')
    
            return paragraphs
    
        except requests.exceptions.RequestException as e:
            print(f"Error fetching the URL: {e}")
    return None

In [6]:
def check_alphabet_ratio(text):
    # Remove newline characters from the string
    
    
    # Define the alphabet characters (lowercase and uppercase)
    alphabet = string.ascii_letters  # a-z and A-Z

    # Count total valid characters (ignoring spaces and punctuation)
    total_chars = len([char for char in text if char.isprintable() and not char.isspace()])

    # Count the number of alphabetic characters
    alphabetic_chars = len([char for char in text if char in alphabet])
    
    # If there are no valid characters, return False
    if total_chars == 0:
        return False
    
    # Calculate the percentage of alphabetic characters
    alphabetic_ratio = (alphabetic_chars / total_chars) * 100
    
    # Check if the percentage is 90% or more
    return alphabetic_ratio >= 95

In [7]:
def majority_uppercase(s):
    # Filter alphabetic characters and calculate the total count
    alpha_chars = [char for char in s if char.isalpha()]
    
    if len(alpha_chars) == 0:  # If there are no alphabetic characters
        return False
    
    # Count the number of uppercase letters
    upper_count = sum(1 for char in alpha_chars if char.isupper())
    
    # Check if uppercase letters are the majority
    return upper_count > len(alpha_chars) / 2


In [ ]:
import nltk
from transformers import BertTokenizer

# Download the sentence tokenizer
nltk.download('punkt')

# Initialize the BERT tokenizer (FinBERT can be substituted here)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

def split_paragraph(paragraph, max_tokens=511):
    # Tokenize the paragraph into sentences
    sentences = nltk.sent_tokenize(paragraph)
    
    chunks = []  # List to store the final chunks
    current_chunk = []  # Temporary list to build up sentences in a chunk
    current_chunk_token_count = 0  # Keep track of tokens in the current chunk

    for sentence in sentences:
        # Tokenize the sentence to count its tokens
        tokenized_sentence = tokenizer.tokenize(sentence)
        token_count = len(tokenized_sentence)

        # If adding this sentence would exceed the token limit, store the current chunk and start a new one
        if current_chunk_token_count + token_count > max_tokens:
            chunks.append(" ".join(current_chunk))  # Append the current chunk as a string
            current_chunk = [sentence]  # Start a new chunk with the current sentence
            current_chunk_token_count = token_count  # Reset token count for the new chunk
        else:
            current_chunk.append(sentence)
            current_chunk_token_count += token_count

    # Add the last chunk if it's non-empty
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks



In [9]:
def get_cleaned_paragraphs(url):
    # Example usage:
    #url = "https://www.sec.gov/Archives/edgar/data/1060219/000091431709000800/form10k-98039_sal.txt"
    paragraphs = get_paragraphs_from_url(url)
    if paragraphs==None:
        return None
    liste_paragraphs=[]
    for p in paragraphs:
        cleaned_p = p.replace('\n', ' ')
        if len(cleaned_p)>0:
            if check_alphabet_ratio(cleaned_p):
                if "." in cleaned_p:
                    if not(majority_uppercase(cleaned_p)):
                        if not(cleaned_p in liste_paragraphs):
                            split_list=split_paragraph(cleaned_p)
                            for p_split in split_list:
                                
          
                                liste_paragraphs.append(p_split)

    return liste_paragraphs

## Run

In [ ]:
# Create the "paragraphs" column with None or empty string values
df["paragraphs"] = None

# Use tqdm to create a progress bar
for i in tqdm(range(len(df)), desc="Processing URLs"):
    df.at[i, "paragraphs"] = str(get_cleaned_paragraphs(df.at[i, "url"]))
    
    # Save the DataFrame every 1000 steps
    if (i + 1) % 100 == 0:
        df.to_csv("data/10_K_with_paragraphs.csv", index=False)
        

# Optionally, save the final DataFrame after the loop completes
df.to_csv("data/10_K_with_paragraphs.csv", index=False)
print("Final DataFrame saved.")

## Test

In [11]:
liste_paragraph=get_cleaned_paragraphs('https://www.sec.gov/Archives/edgar/data/729580/000072958018000006/bel10-k.htm')

In [12]:
for i in liste_paragraph:
    print("")
    print("0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000")
    print("")
    print(i)


0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

Indicate by checkmark whether the registrant is a large accelerated filer, an accelerated filer, a non-accelerated filer, a smaller reporting company or emerging growth company. See the definitions of large accelerated filer, accelerated filer and smaller reporting company in Rule 12b-2 of the Exchange Act.

0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

If an emerging growth company, indicate by check mark if the registrant has elected not to use the extended transition period for complying with any new or revised financial accounting standards provided pursuant to Section 13(a) of the Exchange Act. Indicate by check mark whether the registrant is a shell company (as defined in Rule 12b-2 of the Act).

0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

 Bel Fuse

In [13]:
df["url"].iloc[32]

'https://www.sec.gov/Archives/edgar/data/729580/000072958018000006/bel10-k.htm'